<a href="https://colab.research.google.com/github/khanh2601/tranbaokhanhbkhnhp/blob/master/%C4%90%E1%BB%99ng_h%E1%BB%8Dc_Robot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Động học Robot**

In [ ]:
from sympy import symbols, cos, sin, pi, simplify, pprint, tan, expand_trig, sqrt, trigsimp, atan2
from sympy.matrices import Matrix
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import time
import numpy as np

In [ ]:
def pose(theta, alpha, a, d):

  r11 = cos(theta)
  r12 = -sin(theta) * cos(alpha)
  r13 = sin(theta) * sin(alpha)
  r21 = sin(theta)
  r22 = cos(theta) * cos(alpha)
  r23 = - sin(alpha) * cos(theta)
  r32 = sin(alpha)
  r33 = cos(alpha) 
  x = a * cos(theta)
  y = a * sin(theta)
  z = d 

  T = Matrix([
    [r11, r12, r13, x],
    [r21, r22, r23, y],
    [0.0, r32, r33, z],
    [0.0, 0.0, 0.0, 1]
  ])
  
  T = simplify(T)

  return T

In [ ]:
def forward_kin(q1,q2,q3,q4,q5,q6):
  X = []
  Y = []
  Z = []
  d90 = pi/2
  T01 = pose(q1, d90, a1, d1)
  T0g = T01
  px,py,pz = T0g[0,3], T0g[1,3], T0g[2,3]
  X.append(px)
  Y.append(py)
  Z.append(pz)
  T12 = pose(q2, 0, a2, 0)
  T0g = T0g* T12
  px,py,pz = T0g[0,3], T0g[1,3], T0g[2,3]
  X.append(px)
  Y.append(py)
  Z.append(pz)
  T23 = pose(q3, d90, a3, 0)
  T0g = T0g* T23
  px,py,pz = T0g[0,3], T0g[1,3], T0g[2,3]
  X.append(px)
  Y.append(py)
  Z.append(pz)
  T34 = pose(q4, -d90, 0, d4)
  T0g = T0g* T34
  px,py,pz = T0g[0,3], T0g[1,3], T0g[2,3]
  X.append(px)
  Y.append(py)
  Z.append(pz)
  T45 = pose(q5, d90, 0, 0)
  T0g = T0g* T45
  px,py,pz = T0g[0,3], T0g[1,3], T0g[2,3]
  X.append(px)
  Y.append(py)
  Z.append(pz)
  T56 = pose(q6, 0, 0, 0)
  T0g = T0g* T56
  A06 = T0g
  px,py,pz = T0g[0,3], T0g[1,3], T0g[2,3]
  X.append(px)
  Y.append(py)
  Z.append(pz)
  T6g = pose(0, 0, 0, 0.303)
  #final position and rotation
  T0g = T0g* T6g
  px,py,pz = T0g[0,3], T0g[1,3], T0g[2,3]
  X.append(px)
  Y.append(py)
  Z.append(pz)
  #fig = plt.figure()
  #ax = fig.add_subplot(111,projection = '3d')
  #ax.set_xlabel('x axis')
  #ax.set_ylabel('y axis')
  #ax.set_zlabel('z axis')
  
  X = np.reshape(X,(1,7))
  Y = np.reshape(Y,(1,7))
  Z = np.reshape(Z,(1,7))

  return X,Y,Z

In [ ]:
#------------  Rotation Matrix and Euler Angles -----------
# Calculates Rotation Matrix given euler angles.
def eulerAnglesToRotationMatrix(theta) :
     
    R_x = np.array([[1,         0,                  0                   ],
                    [0,         math.cos(theta[0]), -math.sin(theta[0]) ],
                    [0,         math.sin(theta[0]), math.cos(theta[0])  ]
                    ])
         
         
                     
    R_y = np.array([[math.cos(theta[1]),    0,      math.sin(theta[1])  ],
                    [0,                     1,      0                   ],
                    [-math.sin(theta[1]),   0,      math.cos(theta[1])  ]
                    ])
                 
    R_z = np.array([[math.cos(theta[2]),    -math.sin(theta[2]),    0],
                    [math.sin(theta[2]),    math.cos(theta[2]),     0],
                    [0,                     0,                      1]
                    ])
                     
                     
    R = np.dot(R_z, np.dot( R_y, R_x ))
 
    return R

In [ ]:
# Checks if a matrix is a valid rotation matrix.
# def isRotationMatrix(R) :
#     Rt = np.transpose(R)
#     shouldBeIdentity = np.dot(Rt, R)
#     I = np.identity(3, dtype = R.dtype)
#     n = np.linalg.norm(I - shouldBeIdentity)
#     return n < 1e-6

In [ ]:
# def rotationMatrixToEulerAngles(R) :
 
#     assert(isRotationMatrix(R))
     
#     sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
     
#     singular = sy < 1e-6
 
#     if  not singular :
#         x = math.atan2(R[2,1] , R[2,2])
#         y = math.atan2(-R[2,0], sy)
#         z = math.atan2(R[1,0], R[0,0])
#     else :
#         x = math.atan2(-R[1,2], R[1,1])
#         y = math.atan2(-R[2,0], sy)
#         z = 0
 
#     return np.array([x, y, z])

In [ ]:
def get_hypotenuse(a, b):
  return sqrt(a*a + b*b)

In [ ]:
def get_cosine_law_angle(a, b, c):    
  cos_gamma = (a*a + b*b - c*c) / (2*a*b)
  sin_gamma = sqrt(1 - cos_gamma * cos_gamma)
  gamma = atan2(sin_gamma, cos_gamma)

  return gamma

In [ ]:
def get_wrist_center(gripper_point, R0g, dg = 0.303):

  xu, yu, zu = gripper_point 
    
  nx, ny, nz = R0g[0, 2], R0g[1, 2], R0g[2, 2] #Need to modify
  xw = xu - dg * nx
  yw = yu - dg * ny
  zw = zu - dg * nz 

  return xw, yw, zw

In [ ]:
def get_first_three_angles(wrist_center):

  x, y, z  = wrist_center
    
  # a1, a2, a3 = 
  # d1, d4 = 
  # l = 
  # phi = 
  
  x_prime = get_hypotenuse(x, y)
  mx = x_prime -  a1
  my = z - d1 
  m = get_hypotenuse(mx, my)
  alpha = atan2(my, mx)
  
  gamma = get_cosine_law_angle(l, a2, m)
  beta = get_cosine_law_angle(m, a2, l)
  
  q1 = atan2(y, x)
  q2 = pi/2 - beta - alpha 
  q3 = -(gamma - phi)
    
  return q1, q2, q3

In [ ]:
def get_last_three_angles(R):
#R = A36
#  R36 = Matrix([
    # [-sin(q4)*sin(q6) + cos(q4)*cos(q5)*cos(q6), -sin(q4)*cos(q6) - sin(q6)*cos(q4)*cos(q5), sin(q5)*cos(q4)],
    # [ sin(q4)*cos(q5)*cos(q6) + sin(q6)*cos(q4), -sin(q4)*sin(q6)*cos(q5) + cos(q4)*cos(q6), sin(q4)*sin(q5)],
    # [                          -sin(q5)*cos(q6),                            sin(q5)*sin(q6),         cos(q5)]])
  sin_q4 = R[1, 2]
  cos_q4 = R[0, 2]
    
  sin_q5 = sqrt(R[0, 2]**2 + R[1, 2]**2) 
  cos_q5 = R[2, 2]
    
  sin_q6 = R[2, 1]
  cos_q6 = -R[2, 0] 
  
  q4 = atan2(sin_q4, cos_q4)
  q5 = atan2(sin_q5, cos_q5)
  q6 = atan2(sin_q6, cos_q6)
    
  return q4, q5, q6

In [ ]:
def get_angles(x, y, z, alpha, beta, gamma):
  # input: given position and orientation of the gripper_URDF wrt base frame
  # output: angles q1, q2, q3, q4, q5, q6
    
  gripper_point = x, y, z

  q1, q2, q3, q4, q5, q6 = symbols('q1:7')
  alpha, beta, gamma = symbols('alpha beta gamma', real = True)
  px, py, pz = symbols('px py pz', real = True)

  # Rotation of joint 3 wrt to the base frame interms the first three angles q1, q2, q3
  R03 = Matrix([
    [cos(q2 + q3)*cos(q1),  sin(q1), cos(q1)*sin(q2 + q3)],
    [sin(q1)*cos(q2 + q3), -cos(q1), sin(q1)*sin(q2 + q3)],
    [        sin(q2 + q3),        0,        -cos(q2 + q3)]])

  # Transpose of R03 
  R03T = Matrix([
    [cos(q2 + q3)*cos(q1), sin(q1)*cos(q2 + q3),  sin(q2 + q3)],
    [             sin(q1),             -cos(q1),             0],
    [cos(q1)*sin(q2 + q3), sin(q1)*sin(q2 + q3), -cos(q2 + q3)]])

  # Rotation of joint 6 wrt to frame of joint 3 interms of the last three angles q4, q5, q6
  R36 = Matrix([
    [-sin(q4)*sin(q6) + cos(q4)*cos(q5)*cos(q6), -sin(q4)*cos(q6) - sin(q6)*cos(q4)*cos(q5), sin(q5)*cos(q4)],
    [ sin(q4)*cos(q5)*cos(q6) + sin(q6)*cos(q4), -sin(q4)*sin(q6)*cos(q5) + cos(q4)*cos(q6), sin(q4)*sin(q5)],
    [                          -sin(q5)*cos(q6),                            sin(q5)*sin(q6),         cos(q5)]])

  # Rotation of urdf_gripper with respect to the base frame interms of alpha , beta , gamma 
  R0u = Matrix([
    [                                    cos(beta)*cos(gamma),                                    -cos(beta)*sin(gamma),             sin(beta)],
    [ sin(alpha)*sin(beta)*cos(gamma) + sin(gamma)*cos(alpha), -sin(alpha)*sin(beta)*sin(gamma) + cos(alpha)*cos(gamma), -sin(alpha)*cos(beta)],
    [-cos(alpha)*sin(beta)*cos(gamma) + sin(alpha)*sin(gamma),  cos(alpha)*sin(beta)*sin(gamma) + sin(alpha)*cos(gamma),  cos(alpha)*cos(beta)]])

  # Total transform of gripper wrt to base frame given orientation  (alpha),  (beta),  (beta) and position px, py, pz
  T0g_b = Matrix([
    [                                    cos(beta)*cos(gamma),  -cos(beta)*sin(gamma),                                               sin(beta), px],
    [ sin(alpha)*sin(beta)*cos(gamma) + sin(gamma)*cos(alpha), -sin(alpha)*sin(beta)*sin(gamma) + cos(alpha)*cos(gamma), -sin(alpha)*cos(beta), py],
    [-cos(alpha)*sin(beta)*cos(gamma) + sin(alpha)*sin(gamma),  cos(alpha)*sin(beta)*sin(gamma) + sin(alpha)*cos(gamma),  cos(alpha)*cos(beta), pz],
    [                                                       0,                                                        0,                     0,  1]])


  # Inverse kinematics transformations starts below

  R0u_eval = R0u.evalf(subs = {alpha: alpha, beta: beta, gamma: gamma})
  # R0g_eval = R0u_eval * RguT_eval
  R0g_eval = R0u_eval

  wrist_center = get_wrist_center(gripper_point, R0g_eval, dg = 0.303) #Need to prepare dg

  j1, j2, j3 = get_first_three_angles(wrist_center)

  R03T_eval = R03T.evalf(subs = {q1: j1.evalf(), q2: j2.evalf(), q3: j3.evalf()})
  R36_eval = R03T_eval * R0g_eval

  j4, j5, j6 = get_last_three_angles(R36_eval)

  j1 = j1.evalf()
  j2 = j2.evalf()
  j3 = j3.evalf()
  j4 = j4.evalf()
  j5 = j5.evalf()
  j6 = j6.evalf()

  return j1, j2, j3, j4, j5, j6